In [2]:
# Generate the list of index files archived in EDGAR since start_year (earliest: 2015) until the most recent quarter
import datetime
 
current_year = datetime.date.today().year
current_quarter = (datetime.date.today().month - 1) // 3 + 1
start_year = 2015
years = list(range(start_year, current_year))
quarters = ['QTR1', 'QTR2', 'QTR3', 'QTR4']
history = [(y, q) for y in years for q in quarters]
for i in range(1, current_quarter + 1):
    history.append((current_year, 'QTR%d' % i))
urls = ['https://www.sec.gov/Archives/edgar/full-index/%d/%s/master.idx' % (x[0], x[1]) for x in history]
urls.sort()
 
# Download index files and write content into SQLite
import sqlite3
import requests
 
con = sqlite3.connect('edgar_idx2.db')
cur = con.cursor()
cur.execute('DROP TABLE IF EXISTS idx')
cur.execute('CREATE TABLE idx (cik TEXT, conm TEXT, type TEXT, date TEXT, path TEXT)')
 
for url in urls:
    lines = requests.get(url).text.splitlines()
    records = [tuple(line.split('|')) for line in lines[11:]]
    cur.executemany('INSERT INTO idx VALUES (?, ?, ?, ?, ?)', records)
    print(url, 'downloaded and wrote to SQLite')
 
con.commit()
con.close()
 


https://www.sec.gov/Archives/edgar/full-index/2015/QTR1/master.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2015/QTR2/master.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2015/QTR3/master.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2015/QTR4/master.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2016/QTR1/master.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2016/QTR2/master.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2016/QTR3/master.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2016/QTR4/master.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2017/QTR1/master.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2017/QTR2/master.idx downloaded and wrote to SQLite
https://www.sec.gov/

In [3]:
# Write SQLite database to Stata
import pandas
from sqlalchemy import create_engine
 
engine = create_engine('sqlite:///edgar_idx.db')
with engine.connect() as conn, conn.begin():
    data = pandas.read_sql_table('idx', conn)
    data.to_stata('edgar_idx.dta')